In [1]:
import tensorflow as tf

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import matplotlib.pyplot as plt
import xlrd

In [5]:
# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
from tensorflow.examples.tutorials.mnist import input_data
MNIST = input_data.read_data_sets("stanford-tensorflow-tutorials-master/data/mnist", one_hot=True)

Extracting stanford-tensorflow-tutorials-master/data/mnist/train-images-idx3-ubyte.gz
Extracting stanford-tensorflow-tutorials-master/data/mnist/train-labels-idx1-ubyte.gz
Extracting stanford-tensorflow-tutorials-master/data/mnist/t10k-images-idx3-ubyte.gz
Extracting stanford-tensorflow-tutorials-master/data/mnist/t10k-labels-idx1-ubyte.gz


In [12]:
tf.reset_default_graph()
batch_size = 128
X = tf.placeholder(tf.float32, shape=[batch_size, 784], name='X')
Y = tf.placeholder(tf.float32, shape=[batch_size, 10], name='Y')
W = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.001), name='weights')
b = tf.Variable(tf.zeros([1, 10]), name='bias')
Y_predicted = tf.add(tf.matmul(X, W), b)
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_predicted)
loss = tf.reduce_mean(entropy, name='loss')
saver = tf.train.Saver()
my_global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss, global_step=my_global_step)
with tf.Session() as sess:
    n_batches = int(MNIST.train.num_examples / batch_size)
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        total_loss = 0
        for _ in range(n_batches):
            X_batch, Y_batch = MNIST.train.next_batch(batch_size)
            _, l = sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += l
        if (i % 10 == 0):
            print("loss at epoch", str(i), str(total_loss))
            saver.save(sess, 'checkpoints/mnist', global_step=my_global_step)
    W_value, b_value = sess.run([W, b])
    
    
    
    # test the model
    n_batches = int(MNIST.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = MNIST.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, Y_predicted], feed_dict={X: X_batch, Y:Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # similar to numpy.count_nonzero(boolarray) :(
        total_correct_preds += sess.run(accuracy)
    print("Accuracy {0}".format(total_correct_preds/MNIST.test.num_examples))

loss at epoch 0 899.219835877
loss at epoch 10 359.448229134
loss at epoch 20 275.481594831
loss at epoch 30 240.192964137
loss at epoch 40 219.425388366
loss at epoch 50 208.91089493
loss at epoch 60 198.104617566
loss at epoch 70 189.681468964
loss at epoch 80 182.979230314
loss at epoch 90 179.407201439
Accuracy 0.8979


In [15]:
with tf.Session() as sess:
    saver.restore(sess, 'checkpoints/mnist-39039')
    print(b.eval())

INFO:tensorflow:Restoring parameters from checkpoints/mnist-39039
[[-0.08904406  0.18173683 -0.03771121 -0.06837919  0.05290473  0.20221639
  -0.01894065  0.12670101 -0.30524585 -0.04423724]]


In [21]:
tf.reset_default_graph()
batch_size = 128
X = tf.placeholder(tf.float32, shape=[batch_size, 784], name='X')
Y = tf.placeholder(tf.float32, shape=[batch_size, 10], name='Y')
W = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.001), name='weights')
b = tf.Variable(tf.zeros([1, 10]), name='bias')
Y_predicted = tf.add(tf.matmul(X, W), b)
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_predicted)
loss = tf.reduce_mean(entropy, name='loss')
my_global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss, global_step=my_global_step)
tf.summary.scalar("loss", loss)
tf.summary.histogram("loss_hist", loss)
summary_op = tf.summary.merge_all()
with tf.Session() as sess:
    writer = tf.summary.FileWriter('graphs/', sess.graph)
    n_batches = int(MNIST.train.num_examples / batch_size)
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        total_loss = 0
        for i in range(n_batches):
            X_batch, Y_batch = MNIST.train.next_batch(batch_size)
            _, l, summary = sess.run([optimizer, loss, summary_op], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += l
            writer.add_summary(summary, global_step=i)
        if (i % 10 == 0):
            print("loss at epoch", str(i), str(total_loss))
    W_value, b_value = sess.run([W, b])
    writer.close()
    
    
    
    # test the model
    n_batches = int(MNIST.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = MNIST.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, Y_predicted], feed_dict={X: X_batch, Y:Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # similar to numpy.count_nonzero(boolarray) :(
        total_correct_preds += sess.run(accuracy)
    print("Accuracy {0}".format(total_correct_preds/MNIST.test.num_examples))

Accuracy 0.8978


In [22]:
tf.reset_default_graph()
batch_size = 128
with tf.name_scope("data"):
    X = tf.placeholder(tf.float32, shape=[batch_size, 784], name='X')
    Y = tf.placeholder(tf.float32, shape=[batch_size, 10], name='Y')
with tf.name_scope("weights"):
    W = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.001), name='weights')
    b = tf.Variable(tf.zeros([1, 10]), name='bias')
Y_predicted = tf.add(tf.matmul(X, W), b)
with tf.name_scope("loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_predicted)
    loss = tf.reduce_mean(entropy, name='loss')
my_global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
with tf.name_scope("optimizer"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss, global_step=my_global_step)
tf.summary.scalar("loss", loss)
tf.summary.histogram("loss_hist", loss)
summary_op = tf.summary.merge_all()
with tf.Session() as sess:
    writer = tf.summary.FileWriter('graphs/', sess.graph)
    n_batches = int(MNIST.train.num_examples / batch_size)
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        total_loss = 0
        for i in range(n_batches):
            X_batch, Y_batch = MNIST.train.next_batch(batch_size)
            _, l, summary = sess.run([optimizer, loss, summary_op], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += l
            writer.add_summary(summary, global_step=i)
        if (i % 10 == 0):
            print("loss at epoch", str(i), str(total_loss))
    W_value, b_value = sess.run([W, b])
    writer.close()
    
    
    
    # test the model
    n_batches = int(MNIST.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = MNIST.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, Y_predicted], feed_dict={X: X_batch, Y:Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # similar to numpy.count_nonzero(boolarray) :(
        total_correct_preds += sess.run(accuracy)
    print("Accuracy {0}".format(total_correct_preds/MNIST.test.num_examples))

Accuracy 0.8984


In [24]:
tf.reset_default_graph()
batch_size = 128
with tf.name_scope("data"):
    X = tf.placeholder(tf.float32, shape=[batch_size, 784], name='X')
    Y = tf.placeholder(tf.float32, shape=[batch_size, 10], name='Y')
with tf.name_scope("weights"):
    W = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.001), name='weights')
    b = tf.Variable(tf.zeros([1, 10]), name='bias')
Y_predicted = tf.add(tf.matmul(X, W), b)
with tf.name_scope("loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_predicted)
    loss = tf.reduce_mean(entropy, name='loss')
my_global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
with tf.name_scope("optimizer"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001).minimize(loss, global_step=my_global_step)
tf.summary.scalar("loss", loss)
tf.summary.histogram("loss_hist", loss)
summary_op = tf.summary.merge_all()
with tf.Session() as sess:
    writer = tf.summary.FileWriter('graphs/', sess.graph)
    n_batches = int(MNIST.train.num_examples / batch_size)
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        total_loss = 0
        for j in range(n_batches):
            X_batch, Y_batch = MNIST.train.next_batch(batch_size)
            _, l, summary = sess.run([optimizer, loss, summary_op], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += l
            writer.add_summary(summary, global_step=j)
        if (i % 10 == 0):
            print("loss at epoch", str(i), str(total_loss))
    W_value, b_value = sess.run([W, b])
    writer.close()
    
    
    
    # test the model
    n_batches = int(MNIST.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = MNIST.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, Y_predicted], feed_dict={X: X_batch, Y:Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # similar to numpy.count_nonzero(boolarray) :(
        total_correct_preds += sess.run(accuracy)
    print("Accuracy {0}".format(total_correct_preds/MNIST.test.num_examples))

Accuracy 0.8434


In [26]:
tf.reset_default_graph()
batch_size = 128
with tf.name_scope("data"):
    X = tf.placeholder(tf.float32, shape=[batch_size, 784], name='X')
    Y = tf.placeholder(tf.float32, shape=[batch_size, 10], name='Y')
with tf.name_scope("weights"):
    W = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.001), name='weights')
    b = tf.Variable(tf.zeros([1, 10]), name='bias')
Y_predicted = tf.add(tf.matmul(X, W), b)
with tf.name_scope("loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_predicted)
    loss = tf.reduce_mean(entropy, name='loss')
my_global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
with tf.name_scope("optimizer"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss, global_step=my_global_step)
tf.summary.scalar("loss", loss)
tf.summary.histogram("loss_hist", loss)
summary_op = tf.summary.merge_all()
with tf.Session() as sess:
    writer = tf.summary.FileWriter('graphs/', sess.graph)
    n_batches = int(MNIST.train.num_examples / batch_size)
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        total_loss = 0
        for j in range(n_batches):
            X_batch, Y_batch = MNIST.train.next_batch(batch_size)
            _, l, summary = sess.run([optimizer, loss, summary_op], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += l
            writer.add_summary(summary, global_step=j)
        if (i % 10 == 0):
            print("loss at epoch", str(i), str(total_loss))
    W_value, b_value = sess.run([W, b])
    writer.close()
    
    
    
    # test the model
    n_batches = int(MNIST.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = MNIST.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, Y_predicted], feed_dict={X: X_batch, Y:Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # similar to numpy.count_nonzero(boolarray) :(
        total_correct_preds += sess.run(accuracy)
    print("Accuracy {0}".format(total_correct_preds/MNIST.test.num_examples))

Accuracy 0.8979


In [28]:
tf.reset_default_graph()
batch_size = 128
with tf.name_scope("data"):
    X = tf.placeholder(tf.float32, shape=[batch_size, 784], name='X')
    Y = tf.placeholder(tf.float32, shape=[batch_size, 10], name='Y')
with tf.name_scope("weights"):
    W = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.001), name='weights')
    b = tf.Variable(tf.zeros([1, 10]), name='bias')
Y_predicted = tf.add(tf.matmul(X, W), b)
with tf.name_scope("loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_predicted)
    loss = tf.reduce_mean(entropy, name='loss')
my_global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
with tf.name_scope("optimizer"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss, global_step=my_global_step)
tf.summary.scalar("loss", loss)
tf.summary.histogram("loss_hist", loss)
summary_op = tf.summary.merge_all()
with tf.Session() as sess:
    writer = tf.summary.FileWriter('graphs/', sess.graph)
    n_batches = int(MNIST.train.num_examples / batch_size)
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        total_loss = 0
        for j in range(n_batches):
            X_batch, Y_batch = MNIST.train.next_batch(batch_size)
            _, l, summary = sess.run([optimizer, loss, summary_op], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += l
            writer.add_summary(summary, global_step=j)
        if (i % 100 == 0):
            print("loss at epoch", str(i), str(total_loss))
    W_value, b_value = sess.run([W, b])
    writer.close()
    
    
    
    # test the model
    n_batches = int(MNIST.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = MNIST.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, Y_predicted], feed_dict={X: X_batch, Y:Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # similar to numpy.count_nonzero(boolarray) :(
        total_correct_preds += sess.run(accuracy)
    print("Accuracy {0}".format(total_correct_preds/MNIST.test.num_examples))

loss at epoch 0 900.078745961
loss at epoch 100 175.260608628
loss at epoch 200 153.884535775
loss at epoch 300 144.49419938
loss at epoch 400 138.350453585
loss at epoch 500 133.989421487
loss at epoch 600 130.818318874
loss at epoch 700 129.501750425
loss at epoch 800 127.794463389
loss at epoch 900 124.995478287
Accuracy 0.9187
